In [ ]:
#!/urs/bin/env python3

"""
Codes for loading and preprocessing the data.
"""

import numpy as np
import scipy.io as sio


def load_data(filename, train_evo, test_evo, steps, window_size,
              normalization='none'):
    """
    Load data. Add input pulse profile as the first step.
    The input for the network is 'window_size' times the input profile.

    Parameters
    ----------
    filename : filename as string
    train_evo : number of training evolutions as integer
    test_evo : number of test evolutions as integer
    steps : number of propagation steps as integer
    window_size : RNN window size as integer
    normalization : none (default), max, dBm, manual ...

    Returns
    -------
    i_x : number of grid points (spectral/temporal) as integer
    X_train : training data input, shape (N, window_size, i_x)
    X_test : testing data input, shape (M, window_size, i_x)
    Y_train : training data output, shape (N, i_x)
    Y_test : testing data output, shape (M, i_x)
    """

    # Load data
    mat_contents = sio.loadmat(filename)
    data = mat_contents['data']
    print("data loaded...")
    print(data.shape)

    if normalization == 'none':
        pass
        
    elif normalization == 'max':  # linear scale (normalized)
        m_max = np.max(np.fabs(data))
        print('max:', m_max)
        data = data/m_max

    elif normalization == 'dBm':  # logarithmic scale
        m_max = np.max(np.fabs(data))
        print('max:', m_max)
        data=data/m_max  # normalize
        data = 10*np.log10(data)  # dB scale
        dBlim = 55  # define dynamic range
        data[data < -dBlim] = -dBlim  # set spectrum <-55 to -55
        data = data/dBlim + 1

    elif normalization == 'manual':
        m_max = 10369993.175721595 # SC spectral domain
        print('max:', m_max)
        data = data/m_max


    # the number of grid points
    i_x = data.shape[1]

    # Make the time series
    num_evo = train_evo + test_evo
    evo_size = steps - 1
    num_samples = np.round(num_evo*evo_size).astype(int)
    X_data_series = np.zeros((num_samples, window_size, i_x))
    Y_data_series = np.zeros((num_samples, i_x))

    for evo in range(num_evo):
        evo_data = np.transpose(data[evo, :, :])

        # tile the beginning of the evolution with 'window_size' input profiles
        temp1 = evo_data[0, :]
        temp2 = np.tile(temp1, (window_size - 1, 1))
        evo_data = np.vstack((temp2, evo_data))

        for step in range(evo_size):
            input_data = evo_data[step:step + window_size, :]
            output_data = evo_data[step + window_size, :]
            series_idx = evo*evo_size + step
            X_data_series[series_idx, :, :] = input_data
            Y_data_series[series_idx, :] = output_data


    X_train = X_data_series[:num_samples - test_evo*evo_size]
    X_test = X_data_series[num_samples - test_evo*evo_size:]
    Y_train = Y_data_series[:num_samples - test_evo*evo_size]
    Y_test = Y_data_series[num_samples - test_evo*evo_size:]

    return i_x, X_train, X_test, Y_train, Y_test


def load_data_expt(filename, train_evo, test_evo, steps, window_size,
                   normalization='none'):
    """
    Load data to predict the evolution from a given 'window_size' steps.
    To be used with data sets "HOS_expt_time_151" and "HOS_expt_spec_126".

    Parameters
    ----------
    filename : filename as string
    train_evo : number of training evolutions as integer
    test_evo : number of test evolutions as integer
    steps : number of propagation steps as integer
    window_size : RNN window size as integer
    normalization : none (default), max, manual ...

    Returns
    -------
    i_x : number of grid points (spectral/temporal) as integer
    X_train : training data input, shape (N, window_size, i_x)
    X_test : testing data input, shape (M, window_size, i_x)
    Y_train : training data output, shape (N, i_x)
    Y_test : testing data output, shape (M, i_x)
    """

    # Load data
    mat_contents = sio.loadmat(filename)
    data = mat_contents['data']
    print("data loaded...")
    print(data.shape)

    if normalization == 'none':
        pass

    elif normalization == 'max':  # linear scale (normalized)
        m_max = np.max(np.fabs(data))
        print('max:', m_max)
        data = data/m_max

    elif normalization == 'dBm':  # logarithmic scale
        m_max = np.max(np.fabs(data))
        print('max:', m_max)
        data=data/m_max  # normalize
        data = 10*np.log10(data)  # dB scale
        dBlim = 55  # define dynamic range
        data[data < -dBlim] = -dBlim  # set spectrum <-55 to -55
        data = data/dBlim + 1

    elif normalization == 'manual':
        m_max = 10369993.175721595 # SC spectral domain
        print('max:', m_max)
        data = data/m_max

    # the number of grid points
    i_x = data.shape[1]

    # Make the time series
    num_evo = train_evo+test_evo
    evo_size = steps-window_size
    num_samples = np.round(num_evo*evo_size).astype(int)
    X_data_series = np.zeros((num_samples, window_size, i_x))
    Y_data_series = np.zeros((num_samples, i_x))

    for evo in range(num_evo):
        for step in range(evo_size):
            input_data = np.transpose(data[evo, :, step:step + window_size])
            output_data = data[evo, :, step + window_size]
            series_idx = evo*evo_size + step
            X_data_series[series_idx,:,:] = input_data
            Y_data_series[series_idx,:] = output_data

    X_train = X_data_series[:num_samples - test_evo*evo_size]
    X_test = X_data_series[num_samples - test_evo*evo_size:]
    Y_train = Y_data_series[:num_samples - test_evo*evo_size]
    Y_test = Y_data_series[num_samples - test_evo*evo_size:]

    return i_x, X_train, X_test, Y_train, Y_test


def load_data_addP(filename, train_evo, test_evo, steps, window_size,
                   added_params, normalization='none'):
    """
    Load data. Add input pulse profile as the first step.
    The input for the network is 'window_size' times the input profile.
    The 'added_params' is removed from the network output.

    Parameters
    ----------
    filename : filename as string
    train_evo : number of training evolutions as integer
    test_evo : number of test evolutions as integer
    steps : number of propagation steps as integer
    window_size : RNN window size as integer
    added_params : number of additional parameters as integer
    normalization : none (default), max, maxC, dBmQ, dBmCC, manual ...

    Returns
    -------
    i_x : number of grid points (spectral/temporal) as integer
    X_train : training data input, shape (N, window_size, i_x+added_params)
    X_test : testing data input, shape (M, window_size, i_x+added_params)
    Y_train : training data output, shape (N, i_x)
    Y_test : testing data output, shape (M, i_x)
    """

    # Load data
    mat_contents = sio.loadmat(filename)
    data = mat_contents['data']
    print("data loaded...")
    print(data.shape)

    if normalization == 'none':
        pass

    elif normalization == 'max':  # linear scale (normalized)
        m_max = np.max(np.fabs(data))
        print('max:', m_max)
        data = data/m_max

    elif normalization == 'maxC': # linear scale with chirp parameter
        m_max = np.max(np.fabs(data))
        print('max:', m_max)
        # normalize data
        data[:, added_params:, :] = data[:, added_params:, :]/m_max
        # normalize additional parameters (interval [-8,8] for chirp)
        data[:, :added_params, :] = (data[:, :added_params, :] + 8)/16

    elif normalization == 'dBmQ':  # logarithmic scale with q parameter
        m_max = np.max(np.fabs(data))
        print('max:', m_max)
        # normalize data
        data[:, added_params:, :] = data[:, added_params:, :]/m_max
        data[:, added_params:, :] = 10*np.log10(data[:, added_params:, :]) # dB scale
        dBlim = 55  # define dynamic range
        data[data < -dBlim] = -dBlim  # set spectrum <-55 to -55
        data[:, added_params:, :] = data[:, added_params:, :]/dBlim + 1
        # use interval [1,9] for q parameter
        data[:, :added_params, :] = data[:, :added_params, :]/9

    elif normalization == 'dBmCC':  # logarithmic scale with coupling conditions
        m_max = np.max(np.fabs(data))
        print('max:', m_max)
        # normalize data, coupling conditions are already normalized ([0, 1])
        data[:, added_params:, :] = data[:, added_params:, :]/m_max
        data[:, added_params:, :] = 10*np.log10(data[:, added_params:, :]) # dB scale
        dBlim = 55  # define dynamic range
        data[data < -dBlim] = -dBlim  # set spectrum <-55 to -55
        data[:, added_params:, :] = data[:, added_params:, :]/dBlim + 1

    elif normalization == 'manual':
        m_max = 10369993.175721595 # SC spectral domain
        print('max:', m_max)
        data = data/m_max


    # the number of grid points
    i_x = data.shape[1] - added_params

    # Make the time series
    num_evo = train_evo + test_evo
    evo_size = steps - 1
    num_samples = np.round(num_evo*evo_size).astype(int)
    X_data_series = np.zeros((num_samples, window_size, i_x + added_params))
    Y_data_series = np.zeros((num_samples, i_x))

    for evo in range(num_evo):
        evo_data = np.transpose(data[evo, :, :])

        # tile the beginning of the evolution with 'window_size' input profiles
        temp1 = evo_data[0, :]
        temp2 = np.tile(temp1, (window_size - 1,1))
        evo_data = np.vstack((temp2, evo_data))

        for step in range(evo_size):
            input_data = evo_data[step:step + window_size, :]
            # remove additional parameters from output
            output_data = evo_data[step + window_size, added_params:]
            series_idx = evo*evo_size + step
            X_data_series[series_idx, :, :] = input_data
            Y_data_series[series_idx, :] = output_data

    X_train = X_data_series[:num_samples - test_evo*evo_size]
    X_test = X_data_series[num_samples - test_evo*evo_size:]
    Y_train = Y_data_series[:num_samples - test_evo*evo_size]
    Y_test = Y_data_series[num_samples - test_evo*evo_size:]

    return i_x, X_train, X_test, Y_train, Y_test




---



In [ ]:
#!/urs/bin/env python3

"""
Create and update recurrent neural network.
"""

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, LSTM, GRU
from keras.utils import np_utils
from keras.callbacks import History
from keras import optimizers


from keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional
# def make_RNN_model(window_size, i_x, added_params=0):
#     """
#     Create RNN model

#     Parameters
#     ----------
#     window_size : RNN window size as integer
#     i_x : number of grid points as integer
#     added_params : number of additional parameters as integer (optional)

#     Returns
#     -------
#     model : keras model
#     """

#     # Define model architecture
#     model = Sequential()

#     a = 'relu'
#     input_shape = (window_size, i_x+added_params)

#     # Add 2 GRU layers with dropout
#     model.add(GRU(250, activation=a, input_shape=input_shape, return_sequences=True))
#     model.add(GRU(250, activation=a))
#     model.add(Dropout(0.2))

#     # Add dense layers
#     model.add(Dense(250, activation=a))
#     model.add(Dense(250, activation=a))
#     model.add(Dense(250, activation=a))
#     # Output layer
#     model.add(Dense(i_x, activation='sigmoid'))

#     # Compile model
#     optimizer = optimizers.Adam(lr=1e-4)
#     loss = 'mean_squared_error'
#     model.compile(loss=loss,
#                   optimizer=optimizer,
#                   metrics=['mse', 'mae'])

#     return model

def make_RNN_model(window_size, i_x, added_params=0):
    """
    Create RNN model

    Parameters
    ----------
    window_size : RNN window size as integer
    i_x : number of grid points as integer
    added_params : number of additional parameters as integer (optional)

    Returns
    -------
    model : keras model
    """

    # Define model architecture
    model = Sequential()

    a = 'relu'
    input_shape = (window_size, i_x+added_params)

    model.add(Bidirectional(LSTM(250, activation=a), input_shape=input_shape))
    model.add(Dense(250, activation=a))
    model.add(Dense(250, activation=a))
    model.add(Dense(i_x, activation='sigmoid'))

    # Compile model
    optimizer = optimizers.RMSprop(lr=1e-4, rho=0.9)
    loss = 'mean_squared_error'
    model.compile(loss=loss,
                  optimizer=optimizer,
                  metrics=['mse', 'mae'])

    return model

def update_RNN_model(model):
    """
    Update RNN model: learning rate, loss, etc..

    Parameters
    ----------
    model : keras model

    Returns
    -------
    model : keras model
    """

    # Compile model
    optimizer = optimizers.RMSprop(lr=1e-5)
    loss = 'mean_squared_error'
    model.compile(loss=loss,
                  optimizer=optimizer,
                  metrics=['mse', 'mae'])

    return model




---



In [ ]:
#!/urs/bin/env python3

"""
Functions for predicting the evolution for various input intensity profiles.
"""

import numpy as np
import time


def pred_evo(model, X_test, test_evo, steps, window_size, i_x):
    """
    Predict evolution from a given input step. The input is 'window_size' times
    the given input.

    Parameters
    ----------
    model : Keras model
    X_test: : test data, size (samples, window_size, i_x)
    test_evo : number of test evolutions as integer
    steps : steps as integer
    window_size : RNN window size as integer
    i_x : number of grid points (spectral/temporal) as integer

    Returns
    -------
    Y_submit : results matrix, size (samples, i_x)
    """

    # Make the time series
    evo_size = steps - 1
    Y_submit = np.zeros((test_evo, evo_size, i_x))
    test_data = X_test[::evo_size,:,:]  # select fiber input profiles

    for step in range(evo_size):
        test_result = model.predict(test_data)
        Y_submit[:,step,:] = test_result
        test_result = np.expand_dims(test_result, axis=1)
        test_data = np.concatenate((test_data,test_result), axis=1)
        test_data = test_data[:, 1:, :]

    # reshape to the original dimensions
    Y_submit = np.reshape(Y_submit,(evo_size*test_evo, i_x))

    return Y_submit


def pred_evo_expt(model, X_test, test_evo, steps, window_size, i_x):
    """
    Predict evolution from given 'window_size' number of steps.
    To be used with data sets "HOS_expt_time_151" and "HOS_expt_spec_126".

    Parameters
    ----------
    model : Keras model
    X_test: : test data, size (samples, window_size, i_x)
    test_evo : number of test evolutions as integer
    steps : steps as integer
    window_size : RNN window size as integer
    i_x : number of grid points (spectral/temporal) as integer

    Returns
    -------
    Y_submit : results matrix, size (samples, i_x)
    """

    # Make the time series
    evo_size = steps - window_size
    Y_submit = np.zeros((test_evo, evo_size, i_x))
    test_data = X_test[::evo_size, :, :]  # select fiber input profiles

    for step in range(evo_size):
        test_result = model.predict(test_data)
        Y_submit[:, step, :] = test_result
        test_result = np.expand_dims(test_result, axis=1)
        test_data = np.concatenate((test_data,test_result), axis=1)
        test_data = test_data[:, 1:, :]

    # reshape to the original dimensions
    Y_submit = np.reshape(Y_submit,(evo_size*test_evo, i_x))

    return Y_submit

def pred_evo_addP(model, X_test, test_evo, steps, window_size, added_params, i_x):
    """
    Predict evolution from a given input step. The input is 'window_size' times
    the given input. The 'added_params' is passed in the predictions.

    Parameters
    ----------
    model : Keras model
    X_test: : test data, size (samples, window_size, i_x)
    test_evo : number of test evolutions as integer
    steps : steps as integer
    window_size : RNN window size as integer
    added_params : number of additional parameters as integer
    i_x : number of grid points (spectral/temporal) as integer

    Returns
    -------
    Y_submit : results matrix, size (samples, i_x)
    """

    # Make the time series
    evo_size = steps - 1
    Y_submit = np.zeros((test_evo, evo_size, i_x))
    test_data = X_test[::evo_size, :, :]  # select fiber input profiles

    for step in range(evo_size):
        test_result = model.predict(test_data)
        Y_submit[:, step, :] = test_result
        ap = test_data[:, 0, :added_params]
        # pass the additional variables for the prediction
        test_result = np.concatenate((ap, test_result), axis=1)
        test_result = np.expand_dims(test_result, axis=1)
        test_data = np.concatenate((test_data,test_result), axis=1)
        test_data = test_data[:, 1:, :]

    # reshape to the original dimensions
    Y_submit = np.reshape(Y_submit,(evo_size*test_evo, i_x))

    return Y_submit




---



In [ ]:
import numpy as np
from matplotlib import pyplot as plt

def plot_history(history, timestr, add_time):
    plt.figure()
    plt.xlabel('Epoch')
    plt.plot(history.epoch, np.array(history.history['loss']),
           label='Train Loss')
    plt.plot(history.epoch, np.array(history.history['val_loss']),
           label = 'Val loss')
    plt.legend()
    #plt.show()
    if add_time:
        fname = '/content/drive/MyDrive/RNN/exp/plotdata'+timestr+'.png'
    else:
        fname = '/content/drive/MyDrive/RNN/plotdata.png'
    plt.savefig(fname)




---



In [ ]:
# #!/urs/bin/env python3

# """
# Train RNN model for predicting higher-order soliton and supercontinuum spectral
# and temporal evolutions. Normalized NLSE, GNLSE and multimode GNLSE have been
# added.
# --salmelal--

# Lauri Salmela
# lauri.salmela@tuni.fi
# Tampere University, 2020
# """

# switch = 1 # 0 if LSTM & 1 if GRU
# model_name = ""

# if switch == 0:
# 	model_name = "LSTM"
# else:
# 	model_name = "GRU"

# import numpy as np
# from keras.models import load_model
# import matplotlib
# matplotlib.use('Agg')
# from matplotlib import pyplot as plt
# import scipy.io as sio
# import time
# import sys

# # from load_data import *
# # from make_RNN_model import *
# # from pred_evo import *
# # from utils import plot_history


# if __name__ == '__main__':

# 	np.random.seed(123)  # for reproducibility

# 	print(sys.version)
# 	start = time.time()

# 	add_time = 1  # add time stamp for saved results. Yes (1), No (0)

# 	### training
# 	num_epoch = 50 # 50(original) number of epochs
# 	window_size = 10 # RNN window size

# 	# select data file
# 	#filename = 'simulations/HOS_NLSE_time_145.mat' # train_evo=2900, test_evo=100, steps=101, i_x=145
# 	#filename = 'simulations/HOS_NLSE_spec_126.mat' # train_evo=2900, test_evo=100, steps=101, i_x=145
# 	#filename = 'simulations/HOS_expt_time_151.mat' # train_evo=2899, test_evo=100, steps=110, i_x=151
# 	#filename = 'simulations/HOS_expt_spec_126.mat' # train_evo=2899, test_evo=100, steps=110, i_x=126
# 	#filename = 'simulations/SC_time_276.mat' # train_evo=1250, test_evo=50, steps=200, i_x=276
# 	#filename = 'simulations/SC_spec_251.mat' # train_evo=1250, test_evo=50, steps=200, i_x=251
# 	#filename = 'simulations/norm_NLSE_time_256.mat' # train_evo=950, test_evo=50, steps=101, i_x=256
# 	#filename = 'simulations/norm_NLSE_spec_128.mat' # train_evo=950, test_evo=50, steps=101, i_x=128
# 	filename = '/content/drive/MyDrive/RNN/data.mat'
# 	#filename = 'simulations/chirped_NLSE_time_256.mat' # train_evo=5900, test_evo=100, steps=101, i_x=256, added_params=10
# 	#filename = 'simulations/norm_GNLSE_spec_132.mat' # train_evo=11800, test_evo=200, steps=51, i_x=132, added_params=10
# 	#filename = 'simulations/MMGNLSE_spec_301.mat' # train_evo=950, test_evo=50, steps=100, i_x=256, added_params=25


# 	# define samples for training and testing, and the number of propagation
# 	# steps in the evolution
# 	train_evo, test_evo, steps = 950, 50, 101

# 	# define the number of added parameters for chirped_NLSE_time_256 (10),
# 	# norm_GNLSE_spec_132 (10) and MMGNLSE_spec_301 (25). 0 for other cases.
# 	added_params = 0

# 	# load data
# 	i_x, X_train, X_test, Y_train, Y_test = load_data(filename, train_evo,
# 													  test_evo, steps,
# 													  window_size,'dBm') # max/dBm

# 	# load_data_expt is used with HOS_expt_time_151 and HOS_expt_spec_126
# 	#i_x, X_train, X_test, Y_train, Y_test = load_data_expt(filename, train_evo,
# 	#													   test_evo, steps,
# 	#													   window_size,'max') # max/dBm

# 	# load_data_addP is used with chirped_NLSE_time_256, norm_GNLSE_spec_132
# 	# and MMGNLSE_spec_301
# 	#i_x, X_train, X_test, Y_train, Y_test = load_data_addP(filename, train_evo,
# 	#													   test_evo, steps,
# 	#													   window_size,
# 	#													   added_params, 'maxC') # maxC/dBmQ/dBmCC

# 	print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
# 	print("READY...")

# 	# create new model (0) or load ready model (1)
# 	model_ready = 0

# 	if model_ready == 0:
# 		# Define model architecture
# 		model = make_RNN_model(window_size, i_x)

# 		# with additional parameters
# 		#model = make_RNN_model(window_size, i_x, added_params)

# 	elif model_ready == 1:
# 		# Load saved model
# 		model = load_model('nets/evo.h5')

# 		model = update_RNN_model(model)

# 	model.summary()

# 	### Fit model on training data
# 	history = model.fit(X_train, Y_train,
# 						epochs=num_epoch,
# 						validation_split=0.1,
# 						verbose=2)

# 	end = time.time()
# 	print("Elapsed time: %d seconds." % (end-start))

# 	timestr = time.strftime("%Y%m%d-%H:%M")

# 	plot_history(history, timestr, add_time)

#  	# Save net
# 	if add_time:
# 		fname = '/content/drive/MyDrive/RNN/exp/netsEvo_'+model_name+timestr+'.h5'
# 	else:
# 		fname = '/content/drive/MyDrive/RNN/exp/netsEvo.h5'
# 	model.save(fname)

# 	print("TESTING STEP-WISE...")
# 	Y_submit = model.predict(X_test)

# 	print('saving results...')
# 	if add_time:
# 		fname = '/content/drive/MyDrive/RNN/exp/test_results'+model_name+timestr+'.mat'
# 	else:
# 		fname = '/content/drive/MyDrive/RNN/exp/test_results.mat'
# 	sio.savemat(fname, {'Y_submit':Y_submit, 'Y_test':Y_test, 'steps':steps,
# 						'window_size':window_size})

# 	print("TESTING USING INPUT PROFILE ONLY...")

# 	Y_submit = pred_evo(model, X_test, test_evo, steps, window_size, i_x)

# 	# pred_evo_expt is used with HOS_expt_time_151 and HOS_expt_spec_126
# 	#Y_submit = pred_evo_expt(model, X_test, test_evo, steps, window_size, i_x)

# 	# pred_evo_expt is used with chirped_NLSE_time_256, norm_GNLSE_spec_132
# 	# and MMGNLSE_spec_301
# 	#Y_submit = pred_evo_addP(model, X_test, test_evo, steps, window_size,
# 	#						 added_params, i_x)

# 	print('saving results from start...')
# 	if add_time:
# 		fname = '/content/drive/MyDrive/RNN/exp/full_test_results_'+model_name+timestr+'.mat'
# 	else:
# 		fname = '/content/drive/MyDrive/RNN/exp/full_test_results.mat'
# 	sio.savemat(fname, {'Y_submit':Y_submit, 'Y_test':Y_test, 'steps':steps,
# 						'window_size':window_size})
# 	print('all done')



---



Just paste the model net location and it tests the model on data along with saving full results in .mat file.

In [ ]:
#!/urs/bin/env python3

"""
Make predictions for higher-order soliton and supercontinuum spectral
and temporal evolutions.Normalized NLSE, GNLSE and multimode GNLSE have been
added.
--salmelal--

Lauri Salmela
lauri.salmela@tuni.fi
Tampere University, 2020
"""

import numpy as np
from keras.models import load_model
from matplotlib import pyplot as plt
import scipy.io as sio
import time
import sys

# from load_data import *
# from make_RNN_model import *
# from pred_evo import *
# from utils import plot_history

##########################
# mytime = time of indian timezone
import datetime
now_utc = datetime.datetime.utcnow()
IST = datetime.timezone(datetime.timedelta(hours=5, minutes=30))
now_ist = now_utc.astimezone(IST)
mytime = now_ist.strftime('%H:%M-%Y-%m-%d')
#####################

if __name__ == '__main__':

	np.random.seed(123)  # for reproducibility

	print(sys.version)

	add_time = 1  # add time stamp for saved results. Yes (1), No (0)

	# select data file
	#filename = 'simulations/HOS_NLSE_time_145.mat' # train_evo=2900, test_evo=100, steps=101, i_x=145
	#filename = 'simulations/HOS_NLSE_spec_126.mat' # train_evo=2900, test_evo=100, steps=101, i_x=145
	#filename = 'simulations/HOS_expt_time_151.mat' # train_evo=2899, test_evo=100, steps=110, i_x=151
	#filename = 'simulations/HOS_expt_spec_126.mat' # train_evo=2899, test_evo=100, steps=110, i_x=126
	#filename = 'simulations/SC_time_276.mat' # train_evo=1250, test_evo=50, steps=200, i_x=276
	#filename = 'simulations/SC_spec_251.mat' # train_evo=1250, test_evo=50, steps=200, i_x=251
	#filename = 'simulations/norm_NLSE_time_256.mat' # train_evo=950, test_evo=50, steps=101, i_x=256
	filename = '/content/drive/MyDrive/RNN/data.mat' # train_evo=950, test_evo=50, steps=101, i_x=128
	#
	#filename = 'simulations/chirped_NLSE_time_256.mat' # train_evo=5900, test_evo=100, steps=101, i_x=256, added_params=10
	#filename = 'simulations/norm_GNLSE_spec_132.mat' # train_evo=11800, test_evo=200, steps=51, i_x=132, added_params=10
	#filename = 'simulations/MMGNLSE_spec_301.mat' # train_evo=950, test_evo=50, steps=100, i_x=256, added_params=25


	# define samples for training and testing, and the number of propagation
	# steps in the evolution
	train_evo, test_evo, steps = 950, 50, 101

	# define the number of added parameters for chirped_NLSE_time_256 (10),
	# norm_GNLSE_spec_132 (10) and MMGNLSE_spec_301 (25). 0 for other cases.
	added_params = 0

	window_size = 10 # RNN window size

	# load data
	i_x, X_train, X_test, Y_train, Y_test = load_data(filename, train_evo,
													  test_evo, steps,
													  window_size,'dBm') # max/dBm

	# load_data_expt is used with HOS_expt_time_151 and HOS_expt_spec_126
	#i_x, X_train, X_test, Y_train, Y_test = load_data_expt(filename, train_evo,
	#													   test_evo, steps,
	#													   window_size,'max') # max/dBm

	# load_data_addP is used with chirped_NLSE_time_256, norm_GNLSE_spec_132
	# and MMGNLSE_spec_301
	#i_x, X_train, X_test, Y_train, Y_test = load_data_addP(filename, train_evo,
	#													   test_evo, steps,
	#													   window_size,
	#													   added_params, 'maxC') # maxC/dBmQ/dBmCC

	print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
	print("READY...")
 
	location = '/content/drive/MyDrive/RNN/exp/netsEvo_GRU13:07-2023-04-26.h5'
	detail = "adam+2dense-"
	model_type = "GRU"
	if("LSTM" in location):
		model_type = "LSTM"
	elif("biLSTM" in location):
		model_type = "biLSTM"
		
	#model = load_model('nets/HOS_NLSE_time_145_60e.h5')
	#model = load_model('nets/HOS_NLSE_spec_126_80e.h5')
	#model = load_model('nets/HOS_NLSE_spec_lin_126_120e.h5')
	#model = load_model('nets/HOS_expt_time_151_60e.h5')
	#model = load_model('nets/HOS_expt_spec_126_80e.h5')
	#model = load_model('nets/HOS_expt_spec_lin_126_120e.h5')
	#model = load_model('nets/SC_time_276_120e.h5')
	#model = load_model('nets/SC_spec_251_80e.h5')
	#model = load_model('nets/SC_spec_lin_251_100e.h5')
	#model = load_model('nets/norm_NLSE_time_256_80e.h5')
	model = load_model(location)
	#model = load_model('nets/chirped_NLSE_time_256_80e.h5')
	#model = load_model('nets/norm_GNLSE_spec_132_80e.h5')
	#model = load_model('nets/MMGNLSE_spec_301_80e.h5')

	model.summary()

	timestr = time.strftime("%H:%M--%d-%m-%Y")

	print("TESTING STEP-WISE...")
	Y_submit = model.predict(X_test)

	print('saving results...')
	if add_time:
		fname = '/content/drive/MyDrive/RNN/exp/predictions/'+ model_type + detail + '_results_' +mytime+ '.mat'
	else:
		fname = '/content/drive/MyDrive/RNN/test_results.mat'
	sio.savemat(fname, {'Y_submit':Y_submit, 'Y_test':Y_test, 'steps':steps,
						'window_size':window_size})

	print("TESTING USING INPUT PROFILE ONLY...")

	start = time.time()
	Y_submit = pred_evo(model, X_test, test_evo, steps, window_size, i_x)

	# pred_evo_expt is used with HOS_expt_time_151 and HOS_expt_spec_126
	#Y_submit = pred_evo_expt(model, X_test, test_evo, steps, window_size, i_x)

	# pred_evo_expt is used with chirped_NLSE_time_256, norm_GNLSE_spec_132
	# and MMGNLSE_spec_301
	#Y_submit = pred_evo_addP(model, X_test, test_evo, steps, window_size,
	#						 added_params, i_x)
	end = time.time()

	print("Elapsed time: %f seconds." % (end-start))

	print('saving results from start...')
	if add_time:
		'/content/drive/MyDrive/RNN/exp/predictions/'+ model_type + 'test_results' +mytime+ '.mat'
	else:
		fname = '/content/drive/MyDrive/RNN/full_test_results.mat'
	sio.savemat(fname, {'Y_submit':Y_submit, 'Y_test':Y_test, 'steps':steps,
						'window_size':window_size})
	print(fname + "  " + 'all done' )
  


3.9.16 (main, Dec  7 2022, 01:11:51) 
[GCC 9.4.0]
data loaded...
(1000, 128, 101)
max: 1241553.7064835906
(95000, 10, 128) (5000, 10, 128) (95000, 128) (5000, 128)
READY...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 250)               285000    
                                                                 
 dense (Dense)               (None, 250)               62750     
                                                                 
 dense_1 (Dense)             (None, 128)               32128     
                                                                 
Total params: 379,878
Trainable params: 379,878
Non-trainable params: 0
_________________________________________________________________
TESTING STEP-WISE...
157/157 [==============================] - 2s 11ms/step
saving results...
TESTING USING INPUT PROFILE ONLY...
2/2 [=====



---

